# Setup

Using Colab

## Installs

In [8]:
pip install -U cellxgene-census

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.2/21.2 MB 78.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.4/255.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 98.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 105.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fss

## Imports

In [ ]:
import cellxgene_census
import scanpy as sc
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"

# Data

In [ ]:
census_config = {
    "organism" : "Homo sapiens",
    "var_value_filter" : "feature_type in ['protein_coding']",
    "obs_value_filter" : "sex in ['male', 'female'] and cell_type == 'hepatocyte' and disease == 'normal'",
    "var_column_names" : ["feature_id", "feature_name", "feature_type", "feature_length"],
    "obs_column_names" : ["cell_type", "sex", "assay", "suspension_type"],
}

In [ ]:
with cellxgene_census.open_soma(census_version="2025-01-30") as census:
    adata = cellxgene_census.get_anndata(
        census = census,
        organism = "Homo sapiens",
        var_value_filter = "feature_type in ['protein_coding']",
        var_column_names = ["feature_id", "feature_name", "feature_type", "feature_length"],
        obs_value_filter = "sex in ['male', 'female'] and cell_type == 'hepatocyte' and disease == 'normal'",
        obs_column_names = ["cell_type", "sex", "assay", "suspension_type"],
    )

In [44]:
adata

AnnData object with n_obs × n_vars = 78776 × 20045
    obs: 'cell_type', 'sex', 'disease'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_type', 'feature_length', 'nnz', 'n_measured_obs'

In [47]:
sc.pp.filter_genes(adata, min_cells=1000)
adata

AnnData object with n_obs × n_vars = 78776 × 12851
    obs: 'cell_type', 'sex', 'disease'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_type', 'feature_length', 'nnz', 'n_measured_obs', 'n_cells'

In [48]:
sc.pp.filter_cells(adata, min_genes=200)
adata

AnnData object with n_obs × n_vars = 78339 × 12851
    obs: 'cell_type', 'sex', 'disease', 'n_genes'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_type', 'feature_length', 'nnz', 'n_measured_obs', 'n_cells'

In [49]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [50]:
adata

AnnData object with n_obs × n_vars = 78339 × 12851
    obs: 'cell_type', 'sex', 'disease', 'n_genes'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_type', 'feature_length', 'nnz', 'n_measured_obs', 'n_cells'
    uns: 'log1p'

In [58]:
sc.pp.pca(adata, n_comps=5)

In [60]:
px.scatter(
    x=adata.obsm["X_pca"][:, 0],
    y=adata.obsm["X_pca"][:, 1],
    color=adata.obs["sex"],
    title="PCA of Gene Expression Data",
    labels={"x": "PC1", "y": "PC2"},
    width=600,
    height=600,
)

Next: add Census wrapper to datasets.py